In [1]:
with open('a_example.txt', 'r') as f:
    content = f.read()
    
print(content)

6 2 7
1 2 3 6 5 4
5 2 2
0 1 2 3 4
4 3 1
0 2 3 5



In [5]:
def parseContent(fName):
    with open(fName, 'r') as f:
        content = f.read()
    lines = content.split('\n')
    info = {}
    info['nBooks'] = int(lines[0].split(' ')[0])
    info['nLibraries'] = int(lines[0].split(' ')[1])
    info['nDays'] = int(lines[0].split(' ')[2])
    
    bookScores = {}
    bookScoresLine = lines[1].split(' ')
    for i in range(len(bookScoresLine)):
        bookScores[i] = {'id': i, 'score': int(bookScoresLine[i])}
        
    libraryInfo = {}
    for i in range(info['nLibraries']):
        libraryInfoLine = [int(i) for i in lines[2*i + 2].split(' ')]
        libraryBooksInfoLine = [int(i) for i in lines[2*i + 3].split(' ')]
        currentLibraryInfo = {}
        currentLibraryInfo['nBooks'] = libraryInfoLine[0]
        currentLibraryInfo['signupTime'] = libraryInfoLine[1]
        currentLibraryInfo['shipQty'] = libraryInfoLine[2]
        currentLibraryInfo['books'] = {i for i in libraryBooksInfoLine}
        libraryInfo[i] = currentLibraryInfo
        
    return bookScores, info, libraryInfo

In [6]:
parseContent('a_example.txt')

({0: {'id': 0, 'score': 1},
  1: {'id': 1, 'score': 2},
  2: {'id': 2, 'score': 3},
  3: {'id': 3, 'score': 6},
  4: {'id': 4, 'score': 5},
  5: {'id': 5, 'score': 4}},
 {'nBooks': 6, 'nLibraries': 2, 'nDays': 7},
 {0: {'nBooks': 5, 'signupTime': 2, 'shipQty': 2, 'books': {0, 1, 2, 3, 4}},
  1: {'nBooks': 4, 'signupTime': 3, 'shipQty': 1, 'books': {0, 2, 3, 5}}})

In [7]:
def getOptimalOrder(lib, bookScores):
    def takeScore(el):
        return bookScores[el]['score']
    bookOrder = sorted(lib['books'], key=takeScore, reverse=True)
    return bookOrder

In [8]:
from math import ceil
def getTotalTimeToComplete(lib):
    return lib['signupTime'] + ceil(lib['nBooks']/lib['shipQty'])

In [9]:
def getLibMaxScoreAndOptimalOrder(lib, bookScores, daysToComplete):
    bookOrder = getOptimalOrder(lib, bookScores)
    daysLeft = daysToComplete - lib['signupTime']
    cumulativeScore = 0
    for i in range(daysLeft):
        booksForTheDay = bookOrder[lib['shipQty']*i: lib['shipQty']*i + lib['shipQty']]
        scoreForTheDay = sum([bookScores[i]['score'] for i in booksForTheDay])
        cumulativeScore += scoreForTheDay
    return cumulativeScore, bookOrder

In [118]:
getLibMaxScoreAndOptimalOrder({'nBooks': 4, 'signupTime': 3, 'shipQty': 1, 'books': [0, 2, 3, 5]}, {0: {'id': 0, 'score': 1},
  1: {'id': 1, 'score': 2},
  2: {'id': 2, 'score': 3},
  3: {'id': 3, 'score': 6},
  4: {'id': 4, 'score': 5},
  5: {'id': 5, 'score': 4}}, 7)

(14, [3, 5, 2, 0])

In [147]:
def getAverageBookScore(libInfo, bookScores):
    timeRequired = getTotalTimeToComplete(libInfo)
    totalScore = sum([int(bookScores[bookId]['score']) for bookId in libInfo['books']])
    return totalScore / timeRequired

In [148]:
getAverageBookScore({'nBooks': 4, 'signupTime': 3, 'shipQty': 1, 'books': [0, 2, 3, 5]}, 
{0: {'id': 0, 'score': 1},
  1: {'id': 1, 'score': 2},
  2: {'id': 2, 'score': 3},
  3: {'id': 3, 'score': 6},
  4: {'id': 4, 'score': 5},
  5: {'id': 5, 'score': 4}})

2.0

In [2]:
def generateResult(fName, outName):
    bookScores, info, libraryInfo = parseContent(fName)
    print('Parsed content')
    
    maxDays = info['nDays']
    libMaxScores = {}
    libIds = [libId for (libId, libInfo) in libraryInfo.items()]
    print('Got all library IDS', len(libIds))
    
    def takeAverage(libId):
        return getAverageBookScore(libraryInfo[libId], bookScores)
    sortedLibIdsByAverage = sorted(libIds, key=takeAverage, reverse=True)
    
    counter = 0
    daysPassed = 0
    finalLibIds = []
    for libId in sortedLibIdsByAverage:
        finalLibIds.append(libId)
        currentLibInfo = libraryInfo[libId]
        currentLibMaxScore, currentLibOrder =  getLibMaxScoreAndOptimalOrder(
            currentLibInfo, bookScores, maxDays)
        libMaxScores[libId] = {
            'score': currentLibMaxScore,
            'order': currentLibOrder
        }
        print('{}/{}'.format(counter, len(libIds)))
        counter += 1 
        daysPassed += currentLibInfo['signupTime']
        print('Day: {}'.format(daysPassed))
        if daysPassed > maxDays:
            break
    print('Got scores and order for every library')
        
    nLibs = len(finalLibIds)
    with open(outName, 'w+') as f:
        f.write(str(nLibs) + '\n')
        for libId in finalLibIds:
            f.write('{} {}\n'.format(libId, libraryInfo[libId]['nBooks']))
            f.write(' '.join([str(i) for i in libMaxScores[libId]['order']]) + '\n')
            
    print('Done')

In [3]:
generateResult('a_example.txt', 'example_result.txt')

NameError: name 'parseContent' is not defined

In [155]:
generateResult('b_read_on.txt', 'read_on_result.txt')

Parsed content
Got all library IDS 100
Got optimal order for 1 library
0/100
Got optimal order for 1 library
1/100
Got optimal order for 1 library
2/100
Got optimal order for 1 library
3/100
Got optimal order for 1 library
4/100
Got optimal order for 1 library
5/100
Got optimal order for 1 library
6/100
Got optimal order for 1 library
7/100
Got optimal order for 1 library
8/100
Got optimal order for 1 library
9/100
Got optimal order for 1 library
10/100
Got optimal order for 1 library
11/100
Got optimal order for 1 library
12/100
Got optimal order for 1 library
13/100
Got optimal order for 1 library
14/100
Got optimal order for 1 library
15/100
Got optimal order for 1 library
16/100
Got optimal order for 1 library
17/100
Got optimal order for 1 library
18/100
Got optimal order for 1 library
19/100
Got optimal order for 1 library
20/100
Got optimal order for 1 library
21/100
Got optimal order for 1 library
22/100
Got optimal order for 1 library
23/100
Got optimal order for 1 library
24/

In [156]:
generateResult('c_incunabula.txt', 'c_incunabula_result.txt')

Parsed content
Got all library IDS 10000
Got optimal order for 1 library
0/10000
Got optimal order for 1 library
1/10000
Got optimal order for 1 library
2/10000
Got optimal order for 1 library
3/10000
Got optimal order for 1 library
4/10000
Got optimal order for 1 library
5/10000
Got optimal order for 1 library
6/10000
Got optimal order for 1 library
7/10000
Got optimal order for 1 library
8/10000
Got optimal order for 1 library
9/10000
Got optimal order for 1 library
10/10000
Got optimal order for 1 library
11/10000
Got optimal order for 1 library
12/10000
Got optimal order for 1 library
13/10000
Got optimal order for 1 library
14/10000
Got optimal order for 1 library
15/10000
Got optimal order for 1 library
16/10000
Got optimal order for 1 library
17/10000
Got optimal order for 1 library
18/10000
Got optimal order for 1 library
19/10000
Got optimal order for 1 library
20/10000
Got optimal order for 1 library
21/10000
Got optimal order for 1 library
22/10000
Got optimal order for 1 li

197/10000
Got optimal order for 1 library
198/10000
Got optimal order for 1 library
199/10000
Got optimal order for 1 library
200/10000
Got optimal order for 1 library
201/10000
Got optimal order for 1 library
202/10000
Got optimal order for 1 library
203/10000
Got optimal order for 1 library
204/10000
Got optimal order for 1 library
205/10000
Got optimal order for 1 library
206/10000
Got optimal order for 1 library
207/10000
Got optimal order for 1 library
208/10000
Got optimal order for 1 library
209/10000
Got optimal order for 1 library
210/10000
Got optimal order for 1 library
211/10000
Got optimal order for 1 library
212/10000
Got optimal order for 1 library
213/10000
Got optimal order for 1 library
214/10000
Got optimal order for 1 library
215/10000
Got optimal order for 1 library
216/10000
Got optimal order for 1 library
217/10000
Got optimal order for 1 library
218/10000
Got optimal order for 1 library
219/10000
Got optimal order for 1 library
220/10000
Got optimal order for 1 

396/10000
Got optimal order for 1 library
397/10000
Got optimal order for 1 library
398/10000
Got optimal order for 1 library
399/10000
Got optimal order for 1 library
400/10000
Got optimal order for 1 library
401/10000
Got optimal order for 1 library
402/10000
Got optimal order for 1 library
403/10000
Got optimal order for 1 library
404/10000
Got optimal order for 1 library
405/10000
Got optimal order for 1 library
406/10000
Got optimal order for 1 library
407/10000
Got optimal order for 1 library
408/10000
Got optimal order for 1 library
409/10000
Got optimal order for 1 library
410/10000
Got optimal order for 1 library
411/10000
Got optimal order for 1 library
412/10000
Got optimal order for 1 library
413/10000
Got optimal order for 1 library
414/10000
Got optimal order for 1 library
415/10000
Got optimal order for 1 library
416/10000
Got optimal order for 1 library
417/10000
Got optimal order for 1 library
418/10000
Got optimal order for 1 library
419/10000
Got optimal order for 1 

594/10000
Got optimal order for 1 library
595/10000
Got optimal order for 1 library
596/10000
Got optimal order for 1 library
597/10000
Got optimal order for 1 library
598/10000
Got optimal order for 1 library
599/10000
Got optimal order for 1 library
600/10000
Got optimal order for 1 library
601/10000
Got optimal order for 1 library
602/10000
Got optimal order for 1 library
603/10000
Got optimal order for 1 library
604/10000
Got optimal order for 1 library
605/10000
Got optimal order for 1 library
606/10000
Got optimal order for 1 library
607/10000
Got optimal order for 1 library
608/10000
Got optimal order for 1 library
609/10000
Got optimal order for 1 library
610/10000
Got optimal order for 1 library
611/10000
Got optimal order for 1 library
612/10000
Got optimal order for 1 library
613/10000
Got optimal order for 1 library
614/10000
Got optimal order for 1 library
615/10000
Got optimal order for 1 library
616/10000
Got optimal order for 1 library
617/10000
Got optimal order for 1 

792/10000
Got optimal order for 1 library
793/10000
Got optimal order for 1 library
794/10000
Got optimal order for 1 library
795/10000
Got optimal order for 1 library
796/10000
Got optimal order for 1 library
797/10000
Got optimal order for 1 library
798/10000
Got optimal order for 1 library
799/10000
Got optimal order for 1 library
800/10000
Got optimal order for 1 library
801/10000
Got optimal order for 1 library
802/10000
Got optimal order for 1 library
803/10000
Got optimal order for 1 library
804/10000
Got optimal order for 1 library
805/10000
Got optimal order for 1 library
806/10000
Got optimal order for 1 library
807/10000
Got optimal order for 1 library
808/10000
Got optimal order for 1 library
809/10000
Got optimal order for 1 library
810/10000
Got optimal order for 1 library
811/10000
Got optimal order for 1 library
812/10000
Got optimal order for 1 library
813/10000
Got optimal order for 1 library
814/10000
Got optimal order for 1 library
815/10000
Got optimal order for 1 

990/10000
Got optimal order for 1 library
991/10000
Got optimal order for 1 library
992/10000
Got optimal order for 1 library
993/10000
Got optimal order for 1 library
994/10000
Got optimal order for 1 library
995/10000
Got optimal order for 1 library
996/10000
Got optimal order for 1 library
997/10000
Got optimal order for 1 library
998/10000
Got optimal order for 1 library
999/10000
Got optimal order for 1 library
1000/10000
Got optimal order for 1 library
1001/10000
Got optimal order for 1 library
1002/10000
Got optimal order for 1 library
1003/10000
Got optimal order for 1 library
1004/10000
Got optimal order for 1 library
1005/10000
Got optimal order for 1 library
1006/10000
Got optimal order for 1 library
1007/10000
Got optimal order for 1 library
1008/10000
Got optimal order for 1 library
1009/10000
Got optimal order for 1 library
1010/10000
Got optimal order for 1 library
1011/10000
Got optimal order for 1 library
1012/10000
Got optimal order for 1 library
1013/10000
Got optima

1181/10000
Got optimal order for 1 library
1182/10000
Got optimal order for 1 library
1183/10000
Got optimal order for 1 library
1184/10000
Got optimal order for 1 library
1185/10000
Got optimal order for 1 library
1186/10000
Got optimal order for 1 library
1187/10000
Got optimal order for 1 library
1188/10000
Got optimal order for 1 library
1189/10000
Got optimal order for 1 library
1190/10000
Got optimal order for 1 library
1191/10000
Got optimal order for 1 library
1192/10000
Got optimal order for 1 library
1193/10000
Got optimal order for 1 library
1194/10000
Got optimal order for 1 library
1195/10000
Got optimal order for 1 library
1196/10000
Got optimal order for 1 library
1197/10000
Got optimal order for 1 library
1198/10000
Got optimal order for 1 library
1199/10000
Got optimal order for 1 library
1200/10000
Got optimal order for 1 library
1201/10000
Got optimal order for 1 library
1202/10000
Got optimal order for 1 library
1203/10000
Got optimal order for 1 library
1204/10000


In [157]:
generateResult('d_tough_choices.txt', 'd_tough_choices_result.txt')

Parsed content
Got all library IDS 30000
Got optimal order for 1 library
0/30000
Got optimal order for 1 library
1/30000
Got optimal order for 1 library
2/30000
Got optimal order for 1 library
3/30000
Got optimal order for 1 library
4/30000
Got optimal order for 1 library
5/30000
Got optimal order for 1 library
6/30000
Got optimal order for 1 library
7/30000
Got optimal order for 1 library
8/30000
Got optimal order for 1 library
9/30000
Got optimal order for 1 library
10/30000
Got optimal order for 1 library
11/30000
Got optimal order for 1 library
12/30000
Got optimal order for 1 library
13/30000
Got optimal order for 1 library
14/30000
Got optimal order for 1 library
15/30000
Got optimal order for 1 library
16/30000
Got optimal order for 1 library
17/30000
Got optimal order for 1 library
18/30000
Got optimal order for 1 library
19/30000
Got optimal order for 1 library
20/30000
Got optimal order for 1 library
21/30000
Got optimal order for 1 library
22/30000
Got optimal order for 1 li

198/30000
Got optimal order for 1 library
199/30000
Got optimal order for 1 library
200/30000
Got optimal order for 1 library
201/30000
Got optimal order for 1 library
202/30000
Got optimal order for 1 library
203/30000
Got optimal order for 1 library
204/30000
Got optimal order for 1 library
205/30000
Got optimal order for 1 library
206/30000
Got optimal order for 1 library
207/30000
Got optimal order for 1 library
208/30000
Got optimal order for 1 library
209/30000
Got optimal order for 1 library
210/30000
Got optimal order for 1 library
211/30000
Got optimal order for 1 library
212/30000
Got optimal order for 1 library
213/30000
Got optimal order for 1 library
214/30000
Got optimal order for 1 library
215/30000
Got optimal order for 1 library
216/30000
Got optimal order for 1 library
217/30000
Got optimal order for 1 library
218/30000
Got optimal order for 1 library
219/30000
Got optimal order for 1 library
220/30000
Got optimal order for 1 library
221/30000
Got optimal order for 1 

402/30000
Got optimal order for 1 library
403/30000
Got optimal order for 1 library
404/30000
Got optimal order for 1 library
405/30000
Got optimal order for 1 library
406/30000
Got optimal order for 1 library
407/30000
Got optimal order for 1 library
408/30000
Got optimal order for 1 library
409/30000
Got optimal order for 1 library
410/30000
Got optimal order for 1 library
411/30000
Got optimal order for 1 library
412/30000
Got optimal order for 1 library
413/30000
Got optimal order for 1 library
414/30000
Got optimal order for 1 library
415/30000
Got optimal order for 1 library
416/30000
Got optimal order for 1 library
417/30000
Got optimal order for 1 library
418/30000
Got optimal order for 1 library
419/30000
Got optimal order for 1 library
420/30000
Got optimal order for 1 library
421/30000
Got optimal order for 1 library
422/30000
Got optimal order for 1 library
423/30000
Got optimal order for 1 library
424/30000
Got optimal order for 1 library
425/30000
Got optimal order for 1 

599/30000
Got optimal order for 1 library
600/30000
Got optimal order for 1 library
601/30000
Got optimal order for 1 library
602/30000
Got optimal order for 1 library
603/30000
Got optimal order for 1 library
604/30000
Got optimal order for 1 library
605/30000
Got optimal order for 1 library
606/30000
Got optimal order for 1 library
607/30000
Got optimal order for 1 library
608/30000
Got optimal order for 1 library
609/30000
Got optimal order for 1 library
610/30000
Got optimal order for 1 library
611/30000
Got optimal order for 1 library
612/30000
Got optimal order for 1 library
613/30000
Got optimal order for 1 library
614/30000
Got optimal order for 1 library
615/30000
Got optimal order for 1 library
616/30000
Got optimal order for 1 library
617/30000
Got optimal order for 1 library
618/30000
Got optimal order for 1 library
619/30000
Got optimal order for 1 library
620/30000
Got optimal order for 1 library
621/30000
Got optimal order for 1 library
622/30000
Got optimal order for 1 

802/30000
Got optimal order for 1 library
803/30000
Got optimal order for 1 library
804/30000
Got optimal order for 1 library
805/30000
Got optimal order for 1 library
806/30000
Got optimal order for 1 library
807/30000
Got optimal order for 1 library
808/30000
Got optimal order for 1 library
809/30000
Got optimal order for 1 library
810/30000
Got optimal order for 1 library
811/30000
Got optimal order for 1 library
812/30000
Got optimal order for 1 library
813/30000
Got optimal order for 1 library
814/30000
Got optimal order for 1 library
815/30000
Got optimal order for 1 library
816/30000
Got optimal order for 1 library
817/30000
Got optimal order for 1 library
818/30000
Got optimal order for 1 library
819/30000
Got optimal order for 1 library
820/30000
Got optimal order for 1 library
821/30000
Got optimal order for 1 library
822/30000
Got optimal order for 1 library
823/30000
Got optimal order for 1 library
824/30000
Got optimal order for 1 library
825/30000
Got optimal order for 1 

...

14064/30000
Got optimal order for 1 library
14065/30000
Got optimal order for 1 library
14066/30000
Got optimal order for 1 library
14067/30000
Got optimal order for 1 library
14068/30000
Got optimal order for 1 library
14069/30000
Got optimal order for 1 library
14070/30000
Got optimal order for 1 library
14071/30000
Got optimal order for 1 library
14072/30000
Got optimal order for 1 library
14073/30000
Got optimal order for 1 library
14074/30000
Got optimal order for 1 library
14075/30000
Got optimal order for 1 library
14076/30000
Got optimal order for 1 library
14077/30000
Got optimal order for 1 library
14078/30000
Got optimal order for 1 library
14079/30000
Got optimal order for 1 library
14080/30000
Got optimal order for 1 library
14081/30000
Got optimal order for 1 library
14082/30000
Got optimal order for 1 library
14083/30000
Got optimal order for 1 library
14084/30000
Got optimal order for 1 library
14085/30000
Got optimal order for 1 library
14086/30000
Got optimal order fo

14258/30000
Got optimal order for 1 library
14259/30000
Got optimal order for 1 library
14260/30000
Got optimal order for 1 library
14261/30000
Got optimal order for 1 library
14262/30000
Got optimal order for 1 library
14263/30000
Got optimal order for 1 library
14264/30000
Got optimal order for 1 library
14265/30000
Got optimal order for 1 library
14266/30000
Got optimal order for 1 library
14267/30000
Got optimal order for 1 library
14268/30000
Got optimal order for 1 library
14269/30000
Got optimal order for 1 library
14270/30000
Got optimal order for 1 library
14271/30000
Got optimal order for 1 library
14272/30000
Got optimal order for 1 library
14273/30000
Got optimal order for 1 library
14274/30000
Got optimal order for 1 library
14275/30000
Got optimal order for 1 library
14276/30000
Got optimal order for 1 library
14277/30000
Got optimal order for 1 library
14278/30000
Got optimal order for 1 library
14279/30000
Got optimal order for 1 library
14280/30000
Got optimal order fo

14447/30000
Got optimal order for 1 library
14448/30000
Got optimal order for 1 library
14449/30000
Got optimal order for 1 library
14450/30000
Got optimal order for 1 library
14451/30000
Got optimal order for 1 library
14452/30000
Got optimal order for 1 library
14453/30000
Got optimal order for 1 library
14454/30000
Got optimal order for 1 library
14455/30000
Got optimal order for 1 library
14456/30000
Got optimal order for 1 library
14457/30000
Got optimal order for 1 library
14458/30000
Got optimal order for 1 library
14459/30000
Got optimal order for 1 library
14460/30000
Got optimal order for 1 library
14461/30000
Got optimal order for 1 library
14462/30000
Got optimal order for 1 library
14463/30000
Got optimal order for 1 library
14464/30000
Got optimal order for 1 library
14465/30000
Got optimal order for 1 library
14466/30000
Got optimal order for 1 library
14467/30000
Got optimal order for 1 library
14468/30000
Got optimal order for 1 library
14469/30000
Got optimal order fo

14640/30000
Got optimal order for 1 library
14641/30000
Got optimal order for 1 library
14642/30000
Got optimal order for 1 library
14643/30000
Got optimal order for 1 library
14644/30000
Got optimal order for 1 library
14645/30000
Got optimal order for 1 library
14646/30000
Got optimal order for 1 library
14647/30000
Got optimal order for 1 library
14648/30000
Got optimal order for 1 library
14649/30000
Got optimal order for 1 library
14650/30000
Got optimal order for 1 library
14651/30000
Got optimal order for 1 library
14652/30000
Got optimal order for 1 library
14653/30000
Got optimal order for 1 library
14654/30000
Got optimal order for 1 library
14655/30000
Got optimal order for 1 library
14656/30000
Got optimal order for 1 library
14657/30000
Got optimal order for 1 library
14658/30000
Got optimal order for 1 library
14659/30000
Got optimal order for 1 library
14660/30000
Got optimal order for 1 library
14661/30000
Got optimal order for 1 library
14662/30000
Got optimal order fo

14834/30000
Got optimal order for 1 library
14835/30000
Got optimal order for 1 library
14836/30000
Got optimal order for 1 library
14837/30000
Got optimal order for 1 library
14838/30000
Got optimal order for 1 library
14839/30000
Got optimal order for 1 library
14840/30000
Got optimal order for 1 library
14841/30000
Got optimal order for 1 library
14842/30000
Got optimal order for 1 library
14843/30000
Got optimal order for 1 library
14844/30000
Got optimal order for 1 library
14845/30000
Got optimal order for 1 library
14846/30000
Got optimal order for 1 library
14847/30000
Got optimal order for 1 library
14848/30000
Got optimal order for 1 library
14849/30000
Got optimal order for 1 library
14850/30000
Got optimal order for 1 library
14851/30000
Got optimal order for 1 library
14852/30000
Got optimal order for 1 library
14853/30000
Got optimal order for 1 library
14854/30000
Got optimal order for 1 library
14855/30000
Got optimal order for 1 library
14856/30000
Got optimal order fo

In [1]:
generateResult('e_so_many_books.txt', 'e_so_many_books_result.txt')

NameError: name 'generateResult' is not defined

In [158]:
generateResult('f_libraries_of_the_world.txt', 'f_libraries_of_the_world_result.txt')

Parsed content
Got all library IDS 1000
Got optimal order for 1 library
0/1000
Got optimal order for 1 library
1/1000
Got optimal order for 1 library
2/1000
Got optimal order for 1 library
3/1000
Got optimal order for 1 library
4/1000
Got optimal order for 1 library
5/1000
Got optimal order for 1 library
6/1000
Got optimal order for 1 library
7/1000
Got optimal order for 1 library
8/1000
Got optimal order for 1 library
9/1000
Got optimal order for 1 library
10/1000
Got optimal order for 1 library
11/1000
Got optimal order for 1 library
12/1000
Got optimal order for 1 library
13/1000
Got optimal order for 1 library
14/1000
Got scores and order for every library
Done


In [108]:
bookScores, info, libraryInfo = parseContent('c_incunabula.txt')

In [103]:
import pandas as pd

In [109]:
df = pd.DataFrame.from_dict(libraryInfo)

In [110]:
df

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
nBooks,13,19,18,14,12,16,15,20,12,19,...,20,17,17,16,14,18,16,12,19,14
signupTime,344,966,1000,276,869,105,807,737,278,400,...,824,902,75,336,261,648,736,668,955,664
shipQty,33474,53538,27830,8039,717,17003,65258,46446,67429,1890,...,68688,21280,72514,26072,27232,69743,17734,90986,4483,47542
books,"[45608, 94192, 70546, 4346, 45948, 94356, 6990...","[30294, 64718, 22277, 51943, 75323, 26185, 431...","[99122, 81548, 95109, 7100, 35387, 2833, 88815...","[49233, 54205, 19769, 93556, 4982, 43193, 2341...","[88055, 84490, 39112, 16808, 75782, 24927, 141...","[31929, 32713, 47081, 83777, 14459, 30091, 699...","[47182, 46, 2535, 32000, 78614, 27899, 8469, 7...","[37050, 47086, 22656, 27984, 79184, 70688, 370...","[71854, 47723, 53591, 82683, 44699, 6488, 9995...","[90178, 64402, 32264, 35949, 95375, 91115, 736...",...,"[78087, 18639, 97012, 74101, 90932, 68991, 959...","[19210, 30752, 51502, 9968, 72860, 33383, 9886...","[21906, 65970, 97315, 18310, 19245, 12834, 638...","[79752, 44185, 63913, 13809, 72203, 27075, 551...","[75783, 20708, 66894, 50457, 81303, 33344, 517...","[70893, 46238, 1518, 37631, 49403, 89542, 8636...","[83827, 6242, 85586, 31883, 43344, 6423, 67187...","[10870, 28529, 44061, 69237, 47148, 53944, 855...","[64416, 73064, 53242, 865, 93052, 85931, 17066...","[47547, 47804, 27423, 52506, 25732, 19634, 693..."
